# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [ ]:
# Import your libraries:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In this lab, we will explore a dataset that describes websites with different features and labels them either benign or malicious . We will use supervised learning algorithms to figure out what feature patterns malicious websites are likely to have and use our model to predict malicious websites.

# Challenge 1 - Explore The Dataset

Let's start by exploring the dataset. First load the data file:

In [ ]:
websites = pd.read_csv('../website.csv')

#### Explore the data from an bird's-eye view.

You should already been very familiar with the procedures now so we won't provide the instructions step by step. Reflect on what you did in the previous labs and explore the dataset.

Things you'll be looking for:

* What the dataset looks like?
* What are the data types?
* Which columns contain the features of the websites?
* Which column contains the feature we will predict? What is the code standing for benign vs malicious websites?
* Do we need to transform any of the columns from categorical to ordinal values? If so what are these columns?

Feel free to add additional cells for your explorations. Make sure to comment what you find out.

In [ ]:
# Your code here

**What the dataset looks like?**

In [ ]:
websites.head()

**What are the data types?**

In [ ]:
websites.info()

In [ ]:
websites.dtypes.value_counts()

We should also look at the summary statistics for all the numeric columns

In [ ]:
websites.describe()

**Which columns contain the features of the websites?**

In [ ]:
for col in websites.columns:
    if websites[col].dtype == 'O':
        print(f'Unique values for {col}: {websites[col].unique().shape[0]}')

In [ ]:
for item in websites.SERVER.unique()[:10]:
    print(item)

In [ ]:
for item in websites.WHOIS_COUNTRY.unique()[:10]:
    print(item)

In [ ]:
for item in websites.WHOIS_COUNTRY.unique()[-10:]:
    print(item)

In [ ]:
missing_values = websites.CONTENT_LENGTH.isnull().sum()/websites.shape[0]
print(f'missing values for CONTENT_LENGTH column: \n {missing_values:.2%}')

1. We can see which columns don't contain useful information for our predictions:
   * The `URL` column contains a unique identifier for each row and will not be useful for prediction
   * There seems to be a close relationship between `WHOIS_COUNTRY` and `WHOIS_STATEPRO`. We should consider dropping one of those columns as well.
   * Almost 50% of the data is missing for `CONTENT_LENGTH` column.
   
2. The date columns (`WHOIS_REGDATE` and `WHOIS_UPDATED_DATE`) are the registration and update dates of the website domains. We could drop these columns.

3. `SERVER` and `WHOIS_COUNTRY`. We need to consolidate values. There are missing values ("None") that we need to handle.

**Which column contains the feature we will predict? What is the code standing for benign vs malicious websites?**

Column `Type` is the response variable.

|**benign** | **malicious**|
|-----------|--------------|
|     0     |       1      |

#### Next, evaluate if the columns in this dataset are strongly correlated.

In the Mushroom supervised learning lab we did recently, we mentioned we are concerned if our dataset has strongly correlated columns because if it is the case we need to choose certain ML algorithms instead of others. We need to evaluate this for our dataset now.

Luckily, most of the columns in this dataset are ordinal which makes things a lot easier for us. In the next cells below, evaluate the level of collinearity of the data.

We provide some general directions for you to consult in order to complete this step:

1. You will create a correlation matrix using the numeric columns in the dataset.

1. Create a heatmap using `seaborn` to visualize which columns have high collinearity.

1. Comment on which columns you might need to remove due to high collinearity.

In [ ]:
# Your code here

In [ ]:
def corr_matrix(df):
    corr_matrix = df._get_numeric_data().corr()
    return corr_matrix

In [ ]:
def correlation_plot(corr, title=''):
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    
    plt.subplots(figsize=(15, 10))
    cmap = sns.diverging_palette(6, 255, as_cmap=True)
    
    chart = sns.heatmap(corr, mask=mask, cmap=cmap, center=0, linewidths=0.5, annot=True, fmt='.2f')
    plt.title(title, fontsize=16)
    return chart

In [ ]:
correlation = corr_matrix(websites)

In [ ]:
corr_plot = correlation_plot(correlation)

In [ ]:
# Your comment here

Some variables are strongly correlated so we could drop some of them:

* `NUMBER_SPECIAL_CHARACTERS`
* `SOURCE_APP_PACKETS`
* `REMOTE_APP_PACKETS`
* `SOURCE_APP_BYTES`
* `REMOTE_APP_BYTES`
* `APP_PACKETS`

# Challenge 2 - Remove Column Collinearity.

From the heatmap you created, you should have seen at least 3 columns that can be removed due to high collinearity. Remove these columns from the dataset.

Note that you should remove as few columns as you can. You don't have to remove all the columns at once. But instead, try removing one column, then produce the heatmap again to determine if additional columns should be removed. As long as the dataset no longer contains columns that are correlated for over 90%, you can stop. Also, keep in mind when two columns have high collinearity, you only need to remove one of them but not both.

In the cells below, remove as few columns as you can to eliminate the high collinearity in the dataset. Make sure to comment on your way so that the instructional team can learn about your thinking process which allows them to give feedback. At the end, print the heatmap again.

In [ ]:
# Your code here

In [ ]:
cols_to_drop = [
    'NUMBER_SPECIAL_CHARACTERS','SOURCE_APP_PACKETS', 'REMOTE_APP_PACKETS',
    'SOURCE_APP_BYTES', 'REMOTE_APP_BYTES', 'APP_PACKETS'
]

In [ ]:
correlation_plot(
    corr_matrix(websites.drop(columns=cols_to_drop[:3]+cols_to_drop[4:]))
);

In [ ]:
# Your comment here

Although there are 5 variables that are highly correlated, I prefer to also drop one more (`SOURCE_APP_BYTES`) in order to reduce the number of features and the complexity.

In [ ]:
# Print heatmap again
websites.drop(columns=cols_to_drop, inplace=True)

correlation_plot(
    corr_matrix(websites)
);

# Challenge 3 - Handle Missing Values

The next step would be handling missing values. **We start by examining the number of missing values in each column, which you will do in the next cell.**

In [ ]:
# Your code here

In [ ]:
websites.isnull().sum()

If you remember in the previous labs, we drop a column if the column contains a high proportion of missing values. After dropping those problematic columns, we drop the rows with missing values.

#### In the cells below, handle the missing values from the dataset. Remember to comment the rationale of your decisions.

In [ ]:
# Your code here

In [ ]:
print(f'{websites.CONTENT_LENGTH.isnull().sum() / websites.shape[0]:.2%}')

Almost 50% of the data is missing for `CONTENT_LENGTH` column, thus we can drop it

In [ ]:
websites.drop('CONTENT_LENGTH', axis=1, inplace=True)

There are two rows more we can drop

In [ ]:
websites.dropna(inplace=True)

In [ ]:
# Your comment here

#### Again, examine the number of missing values in each column. 

If all cleaned, proceed. Otherwise, go back and do more cleaning.

In [ ]:
# Examine missing values in each column

In [ ]:
websites.isnull().sum()

# Challenge 4 - Handle `WHOIS_*` Categorical Data

There are several categorical columns we need to handle. These columns are:

* `URL`
* `CHARSET`
* `SERVER`
* `WHOIS_COUNTRY`
* `WHOIS_STATEPRO`
* `WHOIS_REGDATE`
* `WHOIS_UPDATED_DATE`

How to handle string columns is always case by case. Let's start by working on `WHOIS_COUNTRY`. Your steps are:

1. List out the unique values of `WHOIS_COUNTRY`.
1. Consolidate the country values with consistent country codes. For example, the following values refer to the same country and should use consistent country code:
    * `CY` and `Cyprus`
    * `US` and `us`
    * `SE` and `se`
    * `GB`, `United Kingdom`, and `[u'GB'; u'UK']`

#### In the cells below, fix the country values as intructed above.

In [ ]:
# Your code here

1. List out the unique values of `WHOIS_COUNTRY`.

In [ ]:
websites.WHOIS_COUNTRY.unique()

2. Consolidate the country values with consistent country codes

In [ ]:
websites.WHOIS_COUNTRY.replace(
    {'Cyprus': 'CY', 'us':'US', 'se': 'SE',
     'United Kingdom': 'GB', "[u'GB'; u'UK']": 'GB'
    }, inplace=True
)

In [ ]:
websites.WHOIS_COUNTRY.unique()

Since we have fixed the country values, can we convert this column to ordinal now?

Not yet. If you reflect on the previous labs how we handle categorical columns, you probably remember we ended up dropping a lot of those columns because there are too many unique values. Too many unique values in a column is not desirable in machine learning because it makes prediction inaccurate. But there are workarounds under certain conditions. One of the fixable conditions is:

#### If a limited number of values account for the majority of data, we can retain these top values and re-label all other rare values.

The `WHOIS_COUNTRY` column happens to be this case. You can verify it by print a bar chart of the `value_counts` in the next cell to verify:

In [ ]:
# Your code here

In [ ]:
def barchart(df, x, y, length=8, width=14, title=''):
    df = df.sort_values(x, ascending=False)
    plt.figure(figsize=(width, length))
    chart = sns.barplot(data=df, x=x, y=y)
    plt.title(title + '\n', fontsize=16)
    return chart

In [ ]:
freq_table = websites.WHOIS_COUNTRY.value_counts(normalize=True)\
            .reset_index()\
            .rename(columns={'index':'COUNTRY', 'WHOIS_COUNTRY':'COUNTS'})


barchart(freq_table, x='COUNTS', y='COUNTRY');

#### After verifying, now let's keep the top 10 values of the column and re-label other columns with `OTHER`.

In [ ]:
# Your code here

In [ ]:
other_countries = freq_table.COUNTRY.values[10:]
other_countries.shape[0]

In [ ]:
websites['WHOIS_COUNTRY'] = websites.WHOIS_COUNTRY.apply(
    lambda x: 'OTHER' if x in other_countries else x
)

In [ ]:
freq_table10 = websites.WHOIS_COUNTRY.value_counts(normalize=True)\
            .reset_index()\
            .rename(columns={'index':'COUNTRY', 'WHOIS_COUNTRY':'COUNTS'})


barchart(freq_table10, x='COUNTS', y='COUNTRY');

Now since `WHOIS_COUNTRY` has been re-labelled, we don't need `WHOIS_STATEPRO` any more because the values of the states or provinces may not be relevant any more. We'll drop this column.

In addition, we will also drop `WHOIS_REGDATE` and `WHOIS_UPDATED_DATE`. These are the registration and update dates of the website domains. Not of our concerns.

#### In the next cell, drop `['WHOIS_STATEPRO', 'WHOIS_REGDATE', 'WHOIS_UPDATED_DATE']`.

In [ ]:
# Your code here
websites.drop(columns=[
    'WHOIS_STATEPRO', 'WHOIS_REGDATE', 'WHOIS_UPDATED_DATE'
], inplace=True)

# Challenge 5 - Handle Remaining Categorical Data & Convert to Ordinal

Now print the `dtypes` of the data again. Besides `WHOIS_COUNTRY` which we already fixed, there should be 3 categorical columns left: `URL`, `CHARSET`, and `SERVER`.

In [ ]:
# Your code here
websites.dtypes[websites.dtypes == 'O']

#### `URL` is easy. We'll simply drop it because it has too many unique values that there's no way for us to consolidate.

In [ ]:
# Your code here
websites.drop('URL', axis=1, inplace=True)

#### Print the unique value counts of `CHARSET`. You see there are only a few unique values. So we can keep it as it is.

In [ ]:
# Your code here
websites.CHARSET.unique()

In [ ]:
websites['CHARSET'] = websites.CHARSET.apply(str.upper)\
    .replace({'ISO-8859-1':'ISO-8859', 'NONE':'None'})

In [ ]:
websites.CHARSET.unique()

`SERVER` is a little more complicated. Print its unique values and think about how you can consolidate those values.

#### Before you think of your own solution, don't read the instructions that come next.

In [ ]:
# Your code here

In [ ]:
websites.SERVER.unique().shape[0]

In [ ]:
for item in websites.SERVER.unique()[:10]:
    print(item)

![Think Hard](../think-hard.jpg)

In [ ]:
# Your comment here

# We could try to keep characters before the "/" because there are
# few main servers type

Although there are so many unique values in the `SERVER` column, there are actually only 3 main server types: `Microsoft`, `Apache`, and `nginx`. Just check if each `SERVER` value contains any of those server types and re-label them. For `SERVER` values that don't contain any of those substrings, label with `Other`.

At the end, your `SERVER` column should only contain 4 unique values: `Microsoft`, `Apache`, `nginx`, and `Other`.

In [ ]:
# Your code here

In [ ]:
servers = ['MICROSOFT', 'APACHE', 'NGINX']

def clean_server(field):
    name = field.upper()
    if servers[0] in name:
        return servers[0]
    elif servers[1] in name:
        return servers[1]
    elif servers[2] in name:
        return servers[2]
    else:
        return 'OTHER'

In [ ]:
websites['SERVER'] = websites.SERVER.apply(clean_server)

In [ ]:
# Count `SERVER` value counts here
websites.SERVER.value_counts()

OK, all our categorical data are fixed now. **Let's convert them to ordinal data using Pandas' `get_dummies` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)).** Make sure you drop the categorical columns by passing `drop_first=True` to `get_dummies` as we don't need them any more. **Also, assign the data with dummy values to a new variable `website_dummy`.**

In [ ]:
# Your code here
websites_dummy = pd.get_dummies(websites, drop_first=True)

Now, inspect `website_dummy` to make sure the data and types are intended - there shouldn't be any categorical columns at this point.

In [ ]:
# Your code here
websites_dummy.dtypes

# Challenge 6 - Modeling, Prediction, and Evaluation

We'll start off this section by splitting the data to train and test. **Name your 4 variables `X_train`, `X_test`, `y_train`, and `y_test`. Select 80% of the data for training and 20% for testing.**

In [ ]:
from sklearn.model_selection import train_test_split

# Your code here:


#### In this lab, we will try two different models and compare our results.

The first model we will use in this lab is logistic regression. We have previously learned about logistic regression as a classification algorithm. In the cell below, load `LogisticRegression` from scikit-learn and initialize the model.

In [ ]:
# Your code here:



Next, fit the model to our training data. We have already separated our data into 4 parts. Use those in your model.

In [ ]:
# Your code here:



finally, import `confusion_matrix` and `accuracy_score` from `sklearn.metrics` and fit our testing data. Assign the fitted data to `y_pred` and print the confusion matrix as well as the accuracy score

In [ ]:
# Your code here:



What are your thoughts on the performance of the model? Write your conclusions below.

In [ ]:
# Your conclusions here:



#### Our second algorithm is is K-Nearest Neighbors. 

Though is it not required, we will fit a model using the training data and then test the performance of the model using the testing data. Start by loading `KNeighborsClassifier` from scikit-learn and then initializing and fitting the model. We'll start off with a model where k=3.

In [ ]:
# Your code here:



To test your model, compute the predicted values for the testing sample and print the confusion matrix as well as the accuracy score.

In [ ]:
# Your code here:



#### We'll create another K-Nearest Neighbors model with k=5. 

Initialize and fit the model below and print the confusion matrix and the accuracy score.

In [ ]:
# Your code here:



Did you see an improvement in the confusion matrix when increasing k to 5? Did you see an improvement in the accuracy score? Write your conclusions below.

In [ ]:
# Your conclusions here:



# Bonus Challenge - Feature Scaling

Problem-solving in machine learning is iterative. You can improve your model prediction with various techniques (there is a sweetspot for the time you spend and the improvement you receive though). Now you've completed only one iteration of ML analysis. There are more iterations you can conduct to make improvements. In order to be able to do that, you will need deeper knowledge in statistics and master more data analysis techniques. In this bootcamp, we don't have time to achieve that advanced goal. But you will make constant efforts after the bootcamp to eventually get there.

However, now we do want you to learn one of the advanced techniques which is called *feature scaling*. The idea of feature scaling is to standardize/normalize the range of independent variables or features of the data. This can make the outliers more apparent so that you can remove them. This step needs to happen during Challenge 6 after you split the training and test data because you don't want to split the data again which makes it impossible to compare your results with and without feature scaling. For general concepts about feature scaling, click [here](https://en.wikipedia.org/wiki/Feature_scaling). To read deeper, click [here](https://medium.com/greyatom/why-how-and-when-to-scale-your-features-4b30ab09db5e).

In the next cell, attempt to improve your model prediction accuracy by means of feature scaling. A library you can utilize is `sklearn.preprocessing.RobustScaler` ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)). You'll use the `RobustScaler` to fit and transform your `X_train`, then transform `X_test`. You will use logistic regression to fit and predict your transformed data and obtain the accuracy score in the same way. Compare the accuracy score with your normalized data with the previous accuracy data. Is there an improvement?

In [ ]:
# Your code here